In [ ]:
#Start creating the death label file

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
#load the chart events file with death in hospital label

df = pd.read_csv('chart_events_final.csv')

In [ ]:
# Implementing target variable/labeling column for specific time horizon to predict
time_horizon = 1 # experiment with the days you would like to select for your prediction time horizon

# Generate an empty placeholder for the target 'dod_label'
df['DOD_LABEL'] = np.nan

# Iterate pseudonym
for i_cnt in set(df.GROUP_ID.values):
    session_block = df.loc[(df['GROUP_ID'] == i_cnt)]  
    for index, row in session_block.iterrows():
        curr_time = row['REL_DAY']
        
        val_target = session_block.loc[(session_block['REL_DAY'] > curr_time) & (session_block['REL_DAY'] <= curr_time + time_horizon)]['DEATH_IN_HOSPITAL'].values
        print("Val target is")
        print(val_target)
        
        if 1 in val_target:
            #print('Setting 1')
            df.loc[index, 'DOD_LABEL'] = 1
        elif 0 in val_target:
            #print('Setting 0')
            df.loc[index, 'DOD_LABEL'] = 0
        else:
            #print('Setting nan')
            df.loc[index, 'DOD_LABEL'] = np.nan            

In [ ]:
df = df.dropna(subset=['DOD_LABEL'])

In [ ]:
#Save file with death label

df.to_csv('vitals_with_DOD_LABEL.csv', index=False)

In [ ]:
#Read the death label file again for further modifications

df = pd.read_csv('vitals_with_DOD_LABEL.csv')

In [ ]:
#Merge with static parameters like GENDER, AGE etc.

df_adm = pd.read_csv('ADMISSIONS_FILTERED.csv')
df_final = df.merge(df_adm[['GENDER', 'AGE', 'MARITAL_STATUS', 'GROUP_ID']], how = 'left',
                left_on = 'GROUP_ID', right_on = 'GROUP_ID')

df_final = df_final[['SUBJECT_ID', 'HADM_ID', 'GROUP_ID', 'REL_DAY', 'GENDER', 'AGE', 'MARITAL_STATUS',
       'ART Blood Pressure Alarm - High', 'ART Blood Pressure Alarm - Low',
       'Admission Weight (Kg)', 'Admit Wt', 'Arterial BP #2 [Diastolic]',
       'Arterial BP #2 [Systolic]', 'Arterial BP Mean', 'Arterial BP Mean #2',
       'Arterial BP [Diastolic]', 'Arterial BP [Systolic]',
       'Arterial Blood Pressure Alarm - High',
       'Arterial Blood Pressure Alarm - Low',
       'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean',
       'Arterial Blood Pressure systolic', 'Arterial O2 Saturation',
       'Arterial O2 pressure', 'Daily Weight', 'Heart Rate',
       'Heart Rate Alarm - Low', 'Heart rate Alarm - High', 'Height',
       'Manual BP Mean(calc)', 'Manual BP [Diastolic]', 'Manual BP [Systolic]',
       'Manual Blood Pressure Diastolic Left',
       'Manual Blood Pressure Diastolic Right',
       'Manual Blood Pressure Systolic Left',
       'Manual Blood Pressure Systolic Right',
       'O2 Saturation Pulseoxymetry Alarm - High',
       'O2 Saturation Pulseoxymetry Alarm - Low',
       'O2 saturation pulseoxymetry', 'PAO2', 'SpO2', 'SpO2 Alarm [High]',
       'SpO2 Alarm [Low]', 'Temporary Pacemaker Rate', 'Weight Change',
       'LAST_CHART_TIME', 'DEATHTIME', 'DEATH_IN_HOSPITAL', 'DOD_LABEL']]

#Save a file without one hot encoding for reference

df_final.to_csv('MIMIC_FINAL_WITHOUT_ONE_HOT_ENCODING.csv', index=False)

#Perform one hot encoding on categorial data

one_hot_encoded_data = pd.get_dummies(df_final, columns = ['GENDER', 'MARITAL_STATUS'])

#Restrucutre this dataframe for better readability

one_hot_encoded_data = one_hot_encoded_data[['SUBJECT_ID', 'HADM_ID', 'GROUP_ID', 'REL_DAY', 'AGE',
       'GENDER_F', 'GENDER_M', 'MARITAL_STATUS_DIVORCED',
       'MARITAL_STATUS_LIFE PARTNER', 'MARITAL_STATUS_MARRIED',
       'MARITAL_STATUS_SEPARATED', 'MARITAL_STATUS_SINGLE',
       'MARITAL_STATUS_UNKNOWN (DEFAULT)', 'MARITAL_STATUS_WIDOWED',
       'ART Blood Pressure Alarm - High', 'ART Blood Pressure Alarm - Low',
       'Admission Weight (Kg)', 'Admit Wt', 'Arterial BP #2 [Diastolic]',
       'Arterial BP #2 [Systolic]', 'Arterial BP Mean', 'Arterial BP Mean #2',
       'Arterial BP [Diastolic]', 'Arterial BP [Systolic]',
       'Arterial Blood Pressure Alarm - High',
       'Arterial Blood Pressure Alarm - Low',
       'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean',
       'Arterial Blood Pressure systolic', 'Arterial O2 Saturation',
       'Arterial O2 pressure', 'Daily Weight', 'Heart Rate',
       'Heart Rate Alarm - Low', 'Heart rate Alarm - High', 'Height',
       'Manual BP Mean(calc)', 'Manual BP [Diastolic]', 'Manual BP [Systolic]',
       'Manual Blood Pressure Diastolic Left',
       'Manual Blood Pressure Diastolic Right',
       'Manual Blood Pressure Systolic Left',
       'Manual Blood Pressure Systolic Right',
       'O2 Saturation Pulseoxymetry Alarm - High',
       'O2 Saturation Pulseoxymetry Alarm - Low',
       'O2 saturation pulseoxymetry', 'PAO2', 'SpO2', 'SpO2 Alarm [High]',
       'SpO2 Alarm [Low]', 'Temporary Pacemaker Rate', 'Weight Change',
       'LAST_CHART_TIME', 'DEATHTIME', 'DEATH_IN_HOSPITAL', 'DOD_LABEL']]

In [ ]:
#Save dataframe with one hot encoding
one_hot_encoded_data.to_csv('MIMIC_FINAL_WITH_ONE_HOT_ENCODING.csv', index=False)


#Delete unnecessary columns created due to one hot encoding
del one_hot_encoded_data['LAST_CHART_TIME']
del one_hot_encoded_data['DEATHTIME']
del one_hot_encoded_data['DEATH_IN_HOSPITAL']

#Save this dataframe
one_hot_encoded_data.to_csv('MIMIC_FINAL_WITH_ONE_HOT_ENCODING.csv', index=False)